학습 : 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는것 <br>
지표 : 손실함수 > 가장 작게 만드는 가중치 매개변수를 찾는 것  <br>

손실함수 종류 <BR>
    - 1. MSE <BR>
    - 2. CROSS ENTROPY<BR>

In [2]:
import numpy as np

In [8]:
#MSE
def lossft(y,t):
    return 0.5 * np.sum( (y-t)**2) ## 이진분류

In [9]:
t = [0,0,1,0,0,0,0,0,0,0]  #정답 : 2
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
lossft(np.array(y), np.array(t))

0.09750000000000003

In [10]:
# '7'일 확률이 제일 높은경우
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
lossft(np.array(y), np.array(t))


0.5975

책에선 MSE를 구한다고 하는데 왜 n(=10)이 아닌 2로 나누는 걸까<br>
아무튼 MSE는 Classification엔 적합하지 않음.
https://towardsdatascience.com/why-using-mean-squared-error-mse-cost-function-for-binary-classification-is-a-bad-idea-933089e90df7

Cross Entropy Error ( i.e. CEE) <br>
$$E = -\sum_{k} \ t_k log y_k$$
$y_k$ : 신경망 출력 , $t_k$ : 정답 레이블

In [17]:
def cross_entropy_error(y,t): #Cross Entropy 함수구현
    delta = 1e-7
    return -np.sum(t*np.log(y+delta)) # 아주 작은값을 더해서 Inf가 안나오게

t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

cross_entropy_error(np.array(y), t) # -log(0.6)

0.510825457099338

In [18]:
y=[0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] #잘못 예측
cross_entropy_error(np.array(y), t) #예측이 잘못될수록 CEE가 높다.

2.302584092994546

mini batch 학습 : data가 너무 클 때 임의로 n개의 데이터를 뽑아 n개만 훈련시킴 <br> why? ) 전부 다 사용하면 시간이 너무 오래 든다.

데이터가 N개일 때 CEE
$$E = -\frac{1}{N} \sum_{n} \sum_{k} t_k log y_k$$

In [23]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) #28*28 flatten
print(t_train.shape)

(60000, 784)
(60000, 10)


In [28]:
np.random.choice(60000,10) # 60000중 랜덤으로 10개 고름

array([43401,  6375, 58021,  5440, 48894, 29763, 43267, 17946,  9012,
       36499])

In [35]:
#One-Hot Encoding 되어있을 때
def cross_entropy_error(y,t):
    if y.ndim ==1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
        
    batch_size = y.shape[0]
    return -np.sum( t * np.log(y+1e-7)) / batch_size # log가 Inf가 되지않게 작은값을 더해 줌

In [ ]:
#답이 숫자레이블로 되어 있을 때
def cross_entropy_error(y,t):
    if y.ndim ==1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arrange(batch_size), t] + 1e-7)) /batch_size